#### Optimization model
##### Knapsack problem, Implementing Greedy algorithm

In [ ]:
## Helper function for generating all power sets of a given list.
def power_set(input_set):
    n = len(input_set)
    # The number of subsets for a set of size n is 2^n
    total_subsets = 2**n

    # Generate all subsets using binary representation of numbers from 0 to 2^n - 1
    all_subsets = []
    for i in range(total_subsets):
        subset = [input_set[j] for j in range(n) if (i & (1 << j)) > 0]
        all_subsets.append(subset)

    return all_subsets

# Example usage:
input_set = [1, 9, 4]
result = power_set(input_set)
print(result)

In [ ]:
# Problem 2
# A house burglar wants to decide what items to take based on either the value of the items,
# the weights, or the ratio of value to weight
class Item(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.weight = w

    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def getWeight(self):
        return self.weight
    def getDensity(self):
        return self.value/self.weight
    
    def __str__(self):
        return "<" + str(self.name) + ', ' + str(self.value) + ', ' + str(self.weight) + '>'
    
def value(item):
    return item.getValue()

def weightInverse(item):
    return 1/item.getWeight()

def density(item):
    return item.getValue()/item.getWeight()

def greedy(items, maxWeight, keyFunction):
    itemsCopy = sorted(items, key=keyFunction, reverse=True)
    totalValue, totalWeight = 0, 0
    result = []
    for i in range(len(itemsCopy)):
        if (totalWeight + itemsCopy[i].getWeight()) <= maxWeight:
            result.append(itemsCopy[i])
            totalValue += itemsCopy[i].getValue()
            totalWeight += itemsCopy[i].getWeight()
    
    return (result, totalValue)

def buildItems():
    names = ['clock','painting','radio','vase','book','computer']
    values = [175,90,20,50,10,200]
    weights = [10,9,4,2,1,20]
    Items = []
    for i in range(len(values)):
        Items.append(Item(names[i], values[i], weights[i]))
    return Items

###################################
# Implementing the greedy algorithm
###################################
# The essence of a greedy algorithm is making the best (as defined by some metric) 
# local choice at each step.

def testGreedy(items, maxWeight, keyFunction):
    taken, val = greedy(items, maxWeight, keyFunction)
    print('Total value of items taken is: ', val)
    for item in taken:
        print(' ', item)

def testGreedys(maxUnits):
    items = buildItems()
    print('Use greedy by value to fill a backpack of size: ', maxUnits)
    testGreedy(items, maxUnits, value)
    print('\nUse greedy by weight to fill a backpack with size: ', maxUnits)
    testGreedy(items, maxUnits, weightInverse)
    print('\nUse greedy by density to fill a backpack of size: ', maxUnits)
    testGreedy(items, maxUnits, density)

#############################################################
# Implementing brute force algorithm for the previous problem
#############################################################
# The complexity of the brute force algorithm is O(2^n)
# The brute force algorithm gives the optimal global solution,
# whereas the greedy algorithm gives the optimal local solution(based on a metric).

def chooseBest(pset, maxUnits):
    bestVal = 0
    bestSet = None
    for items in pset:
        itemsVal = 0
        itemsWeight = 0
        for item in items:
            itemsVal += item.getValue()
            itemsWeight += item.getWeight()
            if itemsWeight < maxUnits and itemsVal > bestVal:
                bestVal = itemsVal
                bestSet = items
    return (bestVal, bestSet)

def testBest(maxUnits = 20):
    items = buildItems()
    pset = power_set(items)
    val, taken = chooseBest(pset, maxUnits)
    print("Total value of items taken is: ", val)
    for item in taken:
        print(item)


In [ ]:
testGreedys(20)
# Note: from the result of running this block we can see that the greedy solution doesn't always
# give the optimal solution and there can different criteria for finding the optimal solution.

In [ ]:
testBest(20)
# from the results of this block we can see how the brute force algorithm gives the optimal solution

### Decision tress and Dynamic Programming
Dynamic programming is a method for efficiently solving problems that exhibit the characteristics of overlapping subproblems and optimal substructure.

A problem has optimal substructure if a globally optimal solution can be found by combining optimal solutions to local subproblems.

A problem has optimal substructure if a globally optimal solution can be found by combining optimal solutions to local subproblems.

In [ ]:
def fib(n):
    """Assumes n is an int >= 0
    Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

The previous implementation of Fibonacci algorithm is not efficient because for each n value, all of the previous values are calculated over and over again. For this reason dynamic programming offers a solution of recording the results of each run in a dictionary. This is called **Memoziation**.
The following code shows a new Fibonacci implementation that is more efficient. 

In [ ]:
def fastFib(n, memo = {}):
    """Assumes n is an int >= 0, memo used only by recursive calls
    Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

fastFib(120)

The knapsack problem can be solved using dynamic programming by constructing search binary tree.
A rooted binary tree is an acyclic directed graph in which:
* There is exactly one node with no parent. This is called **Root**.
* Each non-root node has exactly one parent.
* Each node has at most two children nodes. A child-less node is called **leaf**.

Each node in the search tree for the 0/1 knapsack problem is labeled with a quadruple that denotes a partial solution to the knapsack problem. The elements of the quadruple are:
* A set of items to be taken,
* The list of items for which a decision has not been made,
* The total value of the items in the set of items to be taken (this is merely an optimization, since the value could be computed from the set), and
* The remaining space in the knapsack. (Again, this is an optimization, since it is merely the difference between the weight allowed and the weight of all the items taken so far.)

##### Implementation strategy:
The tree is built top-down starting with the root. One element is selected from the still-to-be-considered items. If there is room for that item in the knap- sack, a node is constructed that reflects the consequence of choosing to take that item. By convention, we draw that node as the left child. The right child shows the consequences of choosing not to take that item. The process is then applied recursively until either the knapsack is full or there are no more items to consider. Because each edge represents a decision (to take or not to take an item), such trees are called decision trees.